In [4]:
#retina 
%config InlineBackend.figure_format = 'retina'

In [5]:
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from eazy import filters
from astropy.table import Table, vstack
from scipy.interpolate import interp1d

WD = Path('/data1/hbahk/spherex-photoz/spherex-challenge/test_realisticres')
os.chdir(WD)

In [6]:
def calculate_synthetic_flam(lam, flam, filter_lam, filter_trans):
    """Calculate the synthetic magnitude of a spectrum through a given filter."""
    # Interpolate spectrum and filter transmission to a common wavelength grid
    common_lam = np.linspace(max(min(lam), min(filter_lam)), 
                                     min(max(lam), max(filter_lam)), num=1000)
    flam_interp = np.interp(common_lam, lam, flam)
    trans_interp = np.interp(common_lam, filter_lam, filter_trans)

    # Calculate the flux received through the filter
    synthetic_flam = (np.trapz(flam_interp * trans_interp * common_lam, common_lam) / 
                      np.trapz(trans_interp * common_lam, common_lam))
    
    # Pivot wavelength
    pivot_lam = np.sqrt(np.trapz(common_lam * trans_interp, common_lam) /
                        np.trapz(trans_interp / common_lam, common_lam))
    
    return synthetic_flam, pivot_lam

def flam_to_fnu(lam, flam):
    fnu = 3.34e4*lam**2*flam # in Jy
    return fnu * 1e6 # in uJy

def fnu_to_flam(lam, fnu):
    flam = fnu / (3.34e4*lam**2) # fnu in uJy
    return flam * 1e-6 # in erg/s/cm^2/A

def fnu_to_abmag(fnu):
    return -2.5*np.log10(fnu) + 23.9 # fnu in uJy

def abmag_to_fnu(abmag):
    return 10**(-0.4*(abmag - 23.9)) # in uJy

In [6]:
res = filters.FilterFile('test_lvf_ubvjk.res')
filt_name = res.filters[0].name.split()[0]
filt_names = [f.name.split()[0] for f in res.filters]
err_names = [f'ERR_{f}' for f in filt_names]
colnames = np.vstack([filt_names, err_names]).T.flatten()
eznames = []
for i in range(1, res.NFILT+1):
    eznames.extend([f'F{i}', f'E{i}'])
    
translate = Table([colnames, eznames])
translate.write('test.translate', format='ascii.no_header', overwrite=True)

In [ ]:
mev_fullsky = Table.read('../fullsky.csv', names=['wave', 'dfnu'])
mev_deep = Table.read('../deep.csv', names=['wave', 'dfnu'])

mev_fs_interp = interp1d(mev_fullsky['wave'], mev_fullsky['dfnu'], kind='nearest', fill_value='extrapolate')
mev_deep_interp = interp1d(mev_deep['wave'], mev_deep['dfnu'], kind='nearest', fill_value='extrapolate')

scaling_factor = 1e-3
fiducial_redshift = 0.6
def get_synphot(spec, res, scaling_factor, fiducial_redshift, mev_fs_interp,
                mev_deep_interp, colnames, plot=False, seed=101):
    lam = spec['wave'] * (1+fiducial_redshift)
    flam = spec['flux'] * scaling_factor / (1+fiducial_redshift)**2
    fnu = flam_to_fnu(lam, flam) # in uJy

    Nchan = res.NFILT
    pivlam = np.zeros(res.NFILT)
    syn_fnu_fs = np.zeros(res.NFILT)
    syn_fnu_dp = np.zeros(res.NFILT)
    syn_fnu = np.zeros(res.NFILT)
    syn_fnu_error_fs = np.zeros(res.NFILT)
    syn_fnu_error_dp = np.zeros(res.NFILT)
    np.random.seed(seed)
    for i, f in zip(np.arange(res.NFILT), res.filters):
        wave, trans = f.wave, f.throughput
        syn_flam, pivot_lam = calculate_synthetic_flam(lam, flam, wave, trans)
        syn_fnu_anchor = flam_to_fnu(pivot_lam, syn_flam) # in uJy
        
        error_fs = mev_fs_interp(pivot_lam*1e-4)
        # syn_fnu_error_fs[i] = np.sqrt(error_fs**2 + syn_fnu_anchor)
        syn_fnu_error_fs[i] = np.sqrt(error_fs**2)
        error_dp = mev_deep_interp(pivot_lam*1e-4)
        # syn_fnu_error_dp[i] = np.sqrt(error_dp**2 + syn_fnu_anchor)
        syn_fnu_error_dp[i] = np.sqrt(error_dp**2)
        
        # update the synthetic flux
        syn_fnu_fs[i] = np.random.normal(syn_fnu_anchor, syn_fnu_error_fs[i])
        syn_fnu_dp[i] = np.random.normal(syn_fnu_anchor, syn_fnu_error_dp[i])
        pivlam[i] = pivot_lam
        syn_fnu[i] = syn_fnu_anchor
    
    if plot:
        fig = plt.figure(figsize=(6,4))
        ax = fig.add_subplot(111)
        
        xlim = [0.7, 5.2]
        ax.set_xlim(xlim)
        lam_mask = (lam > xlim[0]*1e4) & (lam < xlim[1]*1e4)
        ax.plot(lam[lam_mask]*1e-4, fnu[lam_mask], 'k')
        ylim = ax.get_ylim()
        
        ax.errorbar(pivlam*1e-4, syn_fnu_fs, yerr=syn_fnu_error_fs, fmt='o', c='tab:blue', label='Full-sky', ms=3, capsize=2)
        ax.errorbar(pivlam*1e-4, syn_fnu_dp, yerr=syn_fnu_error_dp, fmt='o', c='tab:red', label='Deep', ms=3, capsize=2)
        ax.set_xlabel('Wavelength [$\mu$m]')
        ax.set_ylabel(r'$f_\nu$ [$\mu$Jy]')
        ax.set_yscale('log')
        print(ylim)
        ax.set_ylim(ylim)
        
        secax = ax.secondary_yaxis('right', functions=(lambda x: -2.5*np.log10(x*1e-6/3631), lambda x: 3631*1e6*10**(-0.4*x)))
        # tick format
        secax.set_ylabel(r'$m_{\rm AB}$')
        # secax.set_ticks([18.5, 18, 17.5, 17, 16.5])
        from matplotlib.ticker import StrMethodFormatter, ScalarFormatter
        # secax.yaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
        secax.yaxis.set_minor_formatter(ScalarFormatter())
        secax.xaxis.set_minor_formatter(ScalarFormatter())
        # secax.ticklabel_format(axis='y', style='plain')


    arr_fs = np.ravel(np.vstack([syn_fnu_fs, syn_fnu_error_fs]).T)
    tab_fs = Table(arr_fs.T, names=colnames)
    tab_fs['z_spec'] = fiducial_redshift
    # tab_fs['id'] = [0]
    # tab_fs.write('test_synphot_bright_fullsky.dat', format='ascii.commented_header', overwrite=True)

    arr_dp = np.ravel(np.vstack([syn_fnu_dp, syn_fnu_error_dp]).T)
    tab_dp = Table(arr_dp.T, names=colnames)
    tab_dp['z_spec'] = fiducial_redshift
    # tab_dp['id'] = [0]
    # tab_dp.write('test_synphot_bright_deep.dat', format='ascii.commented_header', overwrite=True)
    tab_all = vstack([tab_fs, tab_dp])
    
    return vstack([tab_fs, tab_dp])


In [8]:
from multiprocessing import Pool
from tqdm import tqdm
from contextlib import closing

def worker_init():
    # Ignore CTRL+C in the worker process
    import signal
    signal.signal(signal.SIGINT, signal.SIG_IGN)

def process_redshift(args):
    # Unpack arguments
    spec, res, scaling_factor, redshift, mev_fs_interp, mev_deep_interp, colnames = args
    
    # Your processing code
    tab = get_synphot(spec, res, scaling_factor, redshift, mev_fs_interp, mev_deep_interp, colnames, plot=False)
    return tab

n_proc = 12
target_mags = np.arange(17, 23.25, 0.25)
redshifts = np.logspace(-2, np.log10(6), 30)

for i in range(1, 13):
    catalog = Table()
    print(f'Processing tweak_fsps_QSF_12_v3_{i:03d}.dat')
    spec = Table.read(f'../fsps_full/tweak_fsps_QSF_12_v3_{i:03d}.dat', format='ascii.no_header', names=['wave', 'flux'])
    fnu = flam_to_fnu(spec['wave'], spec['flux'])
    medfnu = np.median(fnu)

    for target_mag in target_mags:
        print(f'...Processing target mag {target_mag:.2f}')
        target_fnu = abmag_to_fnu(target_mag)
        scaling_factor = target_fnu / medfnu

        # Create a list of arguments for each redshift
        args_list = [(spec, res, scaling_factor, z, mev_fs_interp, mev_deep_interp, colnames) for z in redshifts]

        # Use multiprocessing to process redshifts in parallel
        with closing(Pool(n_proc, initializer=worker_init)) as pool:
            try:
                for result in tqdm(pool.imap_unordered(process_redshift, args_list), total=len(redshifts)):
                    catalog = vstack([catalog, result])
            except KeyboardInterrupt:
                pool.terminate()
                pool.join()
                print("Canceled by user.")
                raise KeyboardInterrupt
    
    catalog.write(f'fsps_{i:03d}_synphot.dat', format='ascii.commented_header', overwrite=True)

Processing tweak_fsps_QSF_12_v3_001.dat
...Processing target mag 17.00


100%|██████████| 30/30 [00:03<00:00,  8.13it/s]

...Processing target mag 17.25



100%|██████████| 30/30 [00:03<00:00,  8.38it/s]

...Processing target mag 17.50


...


100%|██████████| 30/30 [00:03<00:00,  8.35it/s]

...Processing target mag 22.75



100%|██████████| 30/30 [00:03<00:00,  8.27it/s]

...Processing target mag 23.00



100%|██████████| 30/30 [00:03<00:00,  8.34it/s]


In [10]:
len(np.array(list(redshifts)*len(target_mags)))

750

In [11]:
len(tab)

1500

In [13]:
# n_proc = 12
# target_mags = np.arange(17, 23.25, 0.25)
# redshifts = np.logspace(-2, np.log10(6), 30)
# redshift_array = np.array(list(redshifts)*len(target_mags))
# redshift_col = np.vstack([redshift_array, redshift_array]).T.flatten()
tlist = []
for i in range(1, 13):
    tab = Table.read(f'fsps_{i:03d}_synphot.dat', format='ascii.commented_header')
    # tab['z_spec'] = redshift_col
    tlist.append(tab)
catalog = vstack(tlist)
catalog['id'] = np.arange(len(catalog))
catalog.write('fsps_synphot.dat', format='ascii.commented_header', overwrite=True)